# Lab 2: LangGraph를 사용한 여행 플래너 구축


## 개요


이 실습에서는 LangGraph를 사용하여 간단한 여행 플래너를 만드는 과정을 안내합니다. LangGraph는 언어 모델을 사용하여 상태 기반의 다단계 애플리케이션을 구축하기 위한 라이브러리입니다. 여행 플래너는 사용자 입력을 수집하고 개인화된 여행 일정을 생성하는 대화형 AI 애플리케이션을 구조화하는 방법을 보여줍니다.


#### 이 실습에서 다룰 내용: 

아래의 측면들을 다룰 예정입니다:

- 그래프를 사용한 에이전트 시스템 구축을 위한 LangGraph 구성요소

- '턴 바이 턴' 대화를 위한 단기 및 장기 메모리 소개

## 에이전트 소개


에이전트는 대규모 언어 모델(LLM)을 활용하여 동적이고 자율적인 방식으로 작업을 수행하는 지능형 시스템 또는 구성 요소입니다. 주요 개념은 다음과 같습니다:


### 에이전트란 무엇인가?

1. 단계별 사고: 에이전트는 LLM을 활용하여 체인-오브-쏘트(chain-of-thought) 추론이라고 하는 구조화된 방식으로 문제를 생각하고 추론합니다. 이를 통해 효과적으로 계획을 세우고, 평가하고, 작업을 실행할 수 있습니다.

2. 도구 접근: 에이전트는 의사 결정과 문제 해결 능력을 향상시키기 위해 외부 도구(예: 계산기, 데이터베이스, API)를 활용할 수 있습니다.

3. 메모리 접근: 에이전트는 컨텍스트를 저장하고 검색할 수 있어 시간이 지남에 따라 작업을 수행하고, 사용자 상호작용에 적응하며, 복잡한 워크플로우를 처리할 수 있습니다.


**AI 에이전트의 주요 특징:**


**인식:** 센서나 데이터 입력을 통해 환경으로부터 정보를 수집하는 능력.

**의사결정:** AI 알고리즘을 사용하여 정보를 처리하고 최선의 행동 방침을 결정.

**행동:** 결정을 실행하고 환경이나 사용자와 상호작용하는 능력.

**학습:** 경험과 피드백을 통해 시간이 지남에 따라 성능을 향상시키는 능력.

**자율성:** 지속적인 인간의 개입 없이 어느 정도 독립적으로 운영.

**목표 지향:** 특정 목표나 작업을 향해 작업.


<img src="./images/agents_memory_light.png" width="45%"/>

## 사용 사례 세부 정보


우리의 여행 플래너는 다음과 같은 간단한 3단계 프로세스를 따릅니다:


1. **초기 사용자 입력**: 

   - 애플리케이션은 사용자에게 AI 에이전트의 도움을 받기 위한 여행 계획을 입력하도록 요청합니다.

   - 이 정보는 상태에 저장됩니다.


2. **관심사 입력**:

   - 사용자에게 여행에 대한 관심사를 제공하도록 요청합니다.

   - 이러한 관심사는 상태에 리스트로 저장됩니다.


3. **일정 생성**:

   - 수집된 도시와 관심사를 사용하여 언어 모델을 활용해 개인화된 당일 여행 일정을 생성합니다.

   - 생성된 일정이 사용자에게 제시됩니다.


이러한 단계 간의 흐름은 LangGraph에 의해 관리되며, 이는 상태 전환을 처리하고 각 단계가 올바른 순서로 실행되도록 보장합니다.

### 설정 및 임포트


먼저, 필요한 모듈을 임포트하고 환경을 설정하겠습니다.

In [ ]:
# %pip install -U --no-cache-dir  \
# "langchain==0.3.7" \
# "langchain-aws==0.2.6" \
# "langchain-community==0.3.5" \
# "langchain-text-splitters==0.3.2" \
# "langchainhub==0.1.20" \
# "langgraph==0.2.45" \
# "langgraph-checkpoint==2.0.2" \
# "langgraph-sdk==0.1.35" \
# "langsmith==0.1.140" \
# "pypdf==3.8,<4" \
# "ipywidgets>=7,<8" \
# "matplotlib==3.9.0" \
# "faiss-cpu==1.8.0"


# %pip install -U --no-cache-dir transformers
# %pip install -U --no-cache-dir boto3
# %pip install grandalf==3.1.2

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
from dotenv import load_dotenv
import os
#load_dotenv()

## LangGraph 기본 사항


### 주요 구성 요소


1. **StateGraph**

   - 이 객체는 실행 중에 순회되는 그래프를 캡슐화합니다.

   - 여행 플래너의 흐름을 정의하는 우리 애플리케이션의 핵심입니다.

   - PlannerState는 우리의 계획 프로세스 상태를 나타내는 사용자 정의 타입입니다.


2. **노드**

    - LangGraph에서 노드는 일반적으로 파이썬 함수입니다.

    - 우리의 그래프에서 사용할 두 가지 주요 노드가 있습니다:

        - 에이전트 노드: 어떤 행동을 취할지(있다면) 결정하는 책임이 있습니다.

        - 도구 노드: 이 노드는 각각의 도구를 호출하고 출력을 반환하는 것을 조율합니다. 

3. **엣지**

   - 로직이 어떻게 라우팅되고 그래프가 어떻게 중지되는 것을 결정할 지 정의합니다.

   - 에이전트가 어떻게 작동하고 다른 노드들이 어떻게 서로 통신하는지 정의합니다.

   - 몇 가지 주요 엣지 유형이 있습니다:

        - 일반적인 엣지: 한 노드에서 다음 노드로 직접 이동합니다.

        - 조건부 엣지: 다음에 어떤 노드로 이동할지 결정하기 위해 함수를 호출합니다.

        - 진입점: 사용자 입력이 도착했을 때 먼저 호출할 노드입니다.

        - 조건부 진입점: 사용자 입력이 도착했을 때 먼저 호출할 노드를 결정하기 위해 함수를 호출합니다.


4. **LLM 통합**: 최종 여정을 생성하기 위해 언어 모델을 활용합니다.

5. **메모리 통합**: 대화를 위한 장기 및 단기 메모리 활용

### 에이전트 상태 정의


우리는 에이전트가 동작하는 동안 유지할 상태를 정의할 것입니다. 먼저, 그래프의 [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)를 정의합니다. State 스키마는 그래프의 모든 노드와 엣지에 대한 입력 스키마 역할을 합니다.

In [ ]:
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    itinerary: str
    city: str
    user_message: str

### 언어 모델 및 프롬프트 설정

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_aws import ChatBedrock
import boto3
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.config import RunnableConfig
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.amazon.nova-lite-v1:0"
provider_id = "amazon"


llm = ChatBedrockConverse(
    model=model_id,
    provider=provider_id,
    temperature=0,
    max_tokens=None,
    client=bedrock_client,
)


itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 유용한 여행 도우미입니다. 사용자의 관심사에 기반하여 {city}의 당일 여행 일정을 만드세요.
        다음 지침을 따르세요:
        1. 아래의 채팅 대화와 사용자의 최신 입력을 사용하여 사용자의 관심사를 파악하세요.
        2. 항상 이동 시간과 식사 시간을 고려하세요 - 모든 것을 하는 것이 불가능하다면 그렇다고 말하세요.
        3. 사용자가 연중 특정 시기나 계절을 언급하지 않았다면, {city}의 여름철을 가정하고 이 가정을 응답에 명시하세요.
        4. 사용자가 여행 예산을 언급하지 않았다면, 적절한 금액을 가정하고 이 가정을 응답에 명시하세요.
        5. 구체적인 시간대와 함께 시간순으로 정리된 간단한 일정표를 글머리 기호로 제공하세요."""),
    MessagesPlaceholder("chat_history"),
    ("human", "{user_message}"),
])

### 노드와 엣지 정의


우리는 컴파일하기 전에 ```StateGraph```에 노드, 엣지 및 영구 메모리를 추가하고 있습니다. 

- 사용자 여행 계획

- Bedrock으로 호출

- 하루 여행 계획 생성 

- 계획을 추가하거나 수정하는 기능

In [ ]:
def input_interests(state: PlannerState) -> PlannerState:
    user_message = state['user_message'] 

    if not state.get('messages', None) : state['messages'] = []
    return {
        **state,
    }


def create_itinerary(state: PlannerState) -> PlannerState:
    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], user_message=state['user_message'], chat_history=state['messages']))
    print("\nFinal Itinerary:")
    print(response.content)
    return {
        **state,
        "messages": state['messages'] + [HumanMessage(content=state['user_message']), AIMessage(content=response.content)],
        "itinerary": response.content
    }

### 그래프 생성 및 컴파일


이제 LangGraph 워크플로우를 생성하고 컴파일하겠습니다. 

 

- 먼저, 위에서 정의한 `State` 클래스로 StateGraph를 초기화합니다.

- 그런 다음, 노드와 엣지를 추가합니다.

- 그래프를 시작할 위치를 나타내기 위해 [`START` 노드, 특별한 노드](https://langchain-ai.github.io/langgraph/concepts/low_level/#start-node)를 사용합니다.

- [`END` 노드](https://langchain-ai.github.io/langgraph/concepts/low_level/#end-node)는 종료 노드를 나타내는 특별한 노드입니다. 

In [ ]:
workflow = StateGraph(PlannerState)


workflow.add_node("input_interests", input_interests)
workflow.add_node("create_itinerary", create_itinerary)


workflow.set_entry_point("input_interests")


workflow.add_edge("input_interests", "create_itinerary")
workflow.add_edge("create_itinerary", END)


# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### 그래프 구조 표시


마지막으로, 그래프 구조에 대한 몇 가지 기본 검사를 수행하기 위해 그래프를 [컴파일합니다](https://langchain-ai.github.io/langgraph/concepts/low_level/#compiling-your-graph). [Mermaid 다이어그램](https://github.com/mermaid-js/mermaid)으로 그래프를 시각화할 수 있습니다.

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

### 그래프를 실행하는 함수 정의


그래프를 컴파일할 때, 우리는 이를 LangChain Runnable로 변환하며, 이는 자동으로 입력과 함께 `.invoke()`, `.stream()` 및 `.batch()`를 호출할 수 있게 합니다. 다음 예제에서는 입력과 함께 그래프를 호출하기 위해 `stream()`을 실행합니다.

In [ ]:
def run_travel_planner(user_request: str, config_dict: dict):
    print(f"Current User Request: {user_request}\n")
    init_input = {"user_message": user_request, "city" : "시애틀"}

    for output in app.stream(init_input, config=config_dict, stream_mode="values"):
        pass  # The nodes themselves now handle all printing

### 여행 플래너 예제


- 이를 실행하기 위해 시스템은 프롬프트를 표시하고 활동에 대한 사용자 입력을 요청합니다 

- 우리는 일반적으로 동적이 될 도시 시애틀로 그래프 상태를 초기화했으며 이는 후속 실습에서 볼 수 있습니다

- 보트 타기, 수영과 같은 것을 입력할 수 있습니다

In [ ]:
config = {"configurable": {"thread_id": "1"}}


user_request = "보트와 수영을 포함한 하루 여행 일정을 만들줘. 완전한 계획이 필요해."
run_travel_planner(user_request, config)

#### 그래프 상태를 조작하기 위한 메모리 세이버 활용

- `대화 메시지`가 그래프 상태의 일부이므로 이를 활용할 수 있습니다

- 그러나 그래프 상태는 세션에 연결되는 `thread_id`로 전달되는 `session_id`에 연결됩니다

- 다른 thread id로 요청을 추가하면 이전 `관심사`를 가지지 않는 새 세션이 생성됩니다

- 그러나 이것은 다른 체크포인트 변수도 가지고 있으므로 이 패턴은 `비동기` 워크플로우에 좋습니다

In [ ]:
config = {"configurable": {"thread_id": "1"}}


user_request = "이 일정에 래프팅도 넣어줄래?"
run_travel_planner(user_request, config)

#### 다른 세션으로 실행


이제 이 세션은 이전 대화를 가지지 않을 것이며 수영이 아닌 `래프팅` 관심사로 새로운 여행 계획을 만들 것입니다

In [ ]:
config = {"configurable": {"thread_id": "11"}}


user_request = "일정에 래프팅을 추가해줄래?"
run_travel_planner(user_request, config)

## 메모리

메모리는 'Multi-Turn' 또는 'Multi-Agent' 협업 대화에서 핵심이며, 특히 여러 날에 걸친 대화에서 더욱 중요합니다. 에이전트의 3가지 주요 측면은 다음과 같습니다:
1. 도구
2. 메모리
3. 플래너


### 메모리를 위한 `외부 저장소` 탐색


AI 에이전트에는 단기 및 장기 메모리라는 두 가지 유형의 메모리가 있으며, 아래에서 설명하겠습니다. 
추가 정보는 이 [링크](https://langchain-ai.github.io/langgraph/concepts/memory/#what-is-memory)에서 확인할 수 있습니다.

대화 메모리는 아래 다이어그램으로 설명할 수 있으며, 이는 에이전트가 접근해야 하는 '턴 바이 턴' 대화를 보여주고 장기 메모리로 저장되는 요약을 설명합니다.

<img src="./images/short-vs-long.png" width="45%"/>



#### 외부 `메모리 지속성` 생성

이 섹션에서는 채팅 메시지에 대한 멀티스레드, 멀티세션 지속성을 활용할 것입니다. 이상적으로는 Redis Store 등과 같은 지속성을 활용하여 세션별 메시지를 저장합니다.

##### 메모리 관리
- 우리는 여러 패턴을 가질 수 있습니다 - 각 에이전트가 자체 세션 메모리를 가질 수 있습니다
- 또는 전체 그래프가 결합된 메모리를 가질 수 있으며, 이 경우 각 에이전트는 자체 메모리를 얻게 됩니다

MemorySaver 또는 Store는 네임스페이스나 스레드 ID로 메모리 섹션을 분리하는 개념을 가지고 있으며, 이를 활용하여 1/ 그래프 수준 메시지나 메모리를 사용하거나 2/ 각 에이전트가 저장소의 공간을 통해 자체 메모리를 가지거나 `ReACT 에이전트`에서처럼 자체 저장소를 가질 수 있습니다.

<img src="./images/multi_memory_light.png" width="60%" alt='multi_memory_light.png' /> 



In [ ]:
from langgraph.store.base import BaseStore, Item, Op, Result
from langgraph.store.memory import InMemoryStore
from typing import Any, Iterable, Literal, NamedTuple, Optional, Union, cast

class CustomMemoryStore(BaseStore):

    def __init__(self, ext_store):
        self.store = ext_store

    def get(self, namespace: tuple[str, ...], key: str) -> Optional[Item]:
        return self.store.get(namespace,key)

    def put(self, namespace: tuple[str, ...], key: str, value: dict[str, Any]) -> None:
        return self.store.put(namespace, key, value)
    def batch(self, ops: Iterable[Op]) -> list[Result]:
        return self.store.batch(ops)
    async def abatch(self, ops: Iterable[Op]) -> list[Result]:
        return self.store.abatch(ops)


#### 이러한 저장소의 사용 방법 빠르게 살펴보기

In [ ]:
in_memory_store = CustomMemoryStore(InMemoryStore())
namespace_u = ("chat_messages", "user_id_1")
key_u="user_id_1"
in_memory_store.put(namespace_u, key_u, {"data":["list a"]})
item_u = in_memory_store.get(namespace_u, key_u)
print(item_u.value, item_u.value['data'])

in_memory_store.list_namespaces()

#### 이전과 유사한 그래프 생성 -- 메시지가 외부화되었기 때문에 그래프 상태에는 메시지가 없을 것입니다.

In [ ]:
class PlannerState(TypedDict):
    itinerary: str
    city: str
    user_message: str

In [ ]:
def input_interests(state: PlannerState, config: RunnableConfig, *, store: BaseStore) -> PlannerState:
    user_message = state['user_message'] #input("Your input: ")
    return {
        **state,
    }

def create_itinerary(state: PlannerState, config: RunnableConfig, *, store: BaseStore) -> PlannerState:
    #- get the history from the store
    user_u = f"user_id_{config['configurable']['thread_id']}"
    namespace_u = ("chat_messages", user_u)
    store_item = store.get(namespace=namespace_u, key=user_u)
    chat_history_messages = store_item.value['data'] if store_item else []
    print(user_u,chat_history_messages)

    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], user_message=state['user_message'], chat_history=chat_history_messages))
    print("\nFinal Itinerary:")
    print(response.content)

    #- add back to the store
    store.put(namespace=namespace_u, key=user_u, value={"data":chat_history_messages+[HumanMessage(content=state['user_message']),AIMessage(content=response.content)]})
    
    return {
        **state,
        "itinerary": response.content
    }

In [ ]:
in_memory_store_n = CustomMemoryStore(InMemoryStore())

workflow = StateGraph(PlannerState)

workflow.add_node("input_interests", input_interests)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_interests")

workflow.add_edge("input_interests", "create_itinerary")
workflow.add_edge("create_itinerary", END)


app = workflow.compile(store=in_memory_store_n)

In [ ]:
def run_travel_planner(user_request: str, config_dict: dict):
    print(f"Current User Request: {user_request}\n")
    init_input = {"user_message": user_request,"city" : "Seattle"}

    for output in app.stream(init_input, config=config_dict, stream_mode="values"):
        pass  # The nodes themselves now handle all printing

config = {"configurable": {"thread_id": "1"}}

user_request = "보트와 수영을 포함한 캘리포니아 하루 여행 일정을 만들어줄래? 여행과 식사 시간을 고려한 완전한 계획이 필요해."
run_travel_planner(user_request, config)


In [ ]:
config = {"configurable": {"thread_id": "1"}}

user_request = "이 일정에 래프팅을 추가해줘."
run_travel_planner(user_request, config)

#### 저장소 빠른 살펴보기

채팅 메시지의 기록을 보여줄 것입니다

In [ ]:
print(in_memory_store_n.list_namespaces())
print(in_memory_store_n.get(('chat_messages', 'user_id_1'),'user_id_1').value)

### 마지막으로 각 `에이전트`가 자신만의 메모리를 가지고 있는 개념에 대해 검토합니다

이를 위해 에이전트를 생성할 때 RunnableWithMessageHistory를 활용할 것입니다
- 여기서는 InMemoryChatMessageHistory를 시뮬레이션하기 위해 생성하지만, 프로덕션 사용 사례에서는 외부화될 것입니다
- 이것을 샘플로 사용하세요

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory


# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.amazon.nova-lite-v1:0"
provider_id = "amazon"

chatbedrock_llm = ChatBedrockConverse(
    model=model_id,
    provider=provider_id,
    temperature=0,
    max_tokens=None,
    client=bedrock_client,
    # other params...
)

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 유용한 여행 도우미입니다. 사용자의 관심사에 기반하여 {city}의 당일 여행 일정을 만드세요.
        다음 지침을 따르세요:
        1. 아래의 채팅 대화와 사용자의 최신 입력을 사용하여 사용자의 관심사를 파악하세요.
        2. 항상 이동 시간과 식사 시간을 고려하세요 - 모든 것을 하는 것이 불가능하다면 그렇다고 말하세요.
        3. 사용자가 연중 특정 시기나 계절을 언급하지 않았다면, {city}의 여름철을 가정하고 이 가정을 응답에 명시하세요.
        4. 사용자가 여행 예산을 언급하지 않았다면, 적절한 금액을 가정하고 이 가정을 응답에 명시하세요.
        5. 구체적인 시간대와 함께 시간순으로 정리된 간단한 일정표를 불릿 포인트로 제공하세요."""),
    MessagesPlaceholder("chat_history"),
    ("human", "{user_message}"),
])
chain = itinerary_prompt | chatbedrock_llm 

history = InMemoryChatMessageHistory()

def get_history():
    return history

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)


In [ ]:
class PlannerState(TypedDict):
    itinerary: str
    city: str
    user_message: str

def input_interests(state: PlannerState, config: RunnableConfig, *, store: BaseStore) -> PlannerState:
    user_message = state['user_message'] #input("Your input: ")
    return {
        **state,
    }

def create_itinerary(state: PlannerState, config: RunnableConfig, *, store: BaseStore) -> PlannerState:
    #- each agent manages it's memory
    response = wrapped_chain.invoke({"city": state['city'], "user_message": state['user_message'], "input": state['user_message']} )
    print("\nFinal Itinerary:")
    print(response.content)
    
    return {
        **state,
        "itinerary": response.content
    }

In [ ]:
workflow = StateGraph(PlannerState)

workflow.add_node("input_interests", input_interests)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_interests")

workflow.add_edge("input_interests", "create_itinerary")
workflow.add_edge("create_itinerary", END)


app = workflow.compile()

In [ ]:
def run_travel_planner(user_request: str, config_dict: dict):
    print(f"Current User Request: {user_request}\n")
    init_input = {"user_message": user_request,"city" : "시애틀"}

    for output in app.stream(init_input, config=config_dict, stream_mode="values"):
        pass  # The nodes themselves now handle all printing

config = {"configurable": {"thread_id": "1"}}

user_request = "보트와 수영을 포함한 일정을 만들어줄래. 완전한 계획이 필요해."
run_travel_planner(user_request, config)

In [ ]:
user_request = "이 여정에 래프팅도 추가해줘."
run_travel_planner(user_request, config)

## 결론

LangGraph 구현을 성공적으로 실행했습니다. 이 실습은 LangGraph를 사용하여 간단하면서도 효과적인 여행 플래너를 만드는 방법을 보여줍니다. 우리의 애플리케이션을 상호 연결된 노드의 그래프로 구조화함으로써, 명확한 관심사 분리와 쉽게 수정 가능한 워크플로우를 달성했습니다. 이 접근 방식은 더 복잡한 애플리케이션으로 확장될 수 있으며, AI 기반 대화형 인터페이스에서 그래프 기반 설계의 강력함과 유연성을 보여줍니다.

다음 실습으로 진행해 주세요.